In [ ]:
import os
import time
import numpy as np
import cv2

from sdcdup.utils import get_project_root

%matplotlib inline
%reload_ext autoreload
%autoreload 2

project_root = get_project_root()
train_image_dir = os.path.join(project_root, os.getenv('RAW_DATA_DIR'), 'train_768')
train_tile_dir = os.path.join(project_root, os.getenv('PROCESSED_DATA_DIR'), 'train_256')
img_ids = os.listdir(train_image_dir)
len(img_ids)

## Speed Test

In [ ]:
def read_from_large(img, ij):
    i, j = ij_pairs[ij]
    tile = img[i * 256:(i + 1) * 256, j * 256:(j + 1) * 256, :]
    tile = cv2.cvtColor(tile, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    return tile

def read_from_small(img_id, ij):
    filebase, fileext = img_id.split('.')
    tile_id = f'{filebase}_{ij}.{fileext}'
    tile = cv2.imread(os.path.join(train_tile_dir, tile_id))
    tile = cv2.cvtColor(tile, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    return tile

def from_large(img_id, ij, kl):
    img = cv2.imread(os.path.join(train_image_dir, img_id))
    tile1 = read_from_large(img, ij)
    tile2 = read_from_large(img, kl)
    return np.dstack([tile1, tile2])

def from_small(img_id, ij, kl):
    tile1 = read_from_small(img_id, ij)
    tile2 = read_from_small(img_id, kl)
    return np.dstack([tile1, tile2])

def from_both(img_id, ij, kl):
    img = cv2.imread(os.path.join(train_image_dir, img_id))
    tile1 = read_from_large(img, ij)
    tile2 = read_from_small(img_id, kl)
    return np.dstack([tile1, tile2])

n_steps = 500

ij_pairs = ((0, 0), (0, 1), (0, 2),
            (1, 0), (1, 1), (1, 2),
            (2, 0), (2, 1), (2, 2))
ijkl = []
for i in range(n_steps):
    ij, kl = np.random.choice(len(ij_pairs), 2, replace=False)
    ijkl.append((ij, kl))

img_id_list = np.random.choice(img_ids, n_steps)
t0 = time.time()
for (ij, kl), img_id in zip(ijkl, img_id_list):
    _ = from_large(img_id, ij, kl)
print('from_large:', time.time() - t0)

img_id_list = np.random.choice(img_ids, n_steps)
t0 = time.time()
for (ij, kl), img_id in zip(ijkl, img_id_list):
    _ = from_small(img_id, ij, kl)
print('from_small:', time.time() - t0)

img_id_list = np.random.choice(img_ids, n_steps)
t0 = time.time()
for (ij, kl), img_id in zip(ijkl, img_id_list):
    _ = from_both(img_id, ij, kl)
print('from_both: ', time.time() - t0)